# Binding site similarity and off-target prediction

Authors:

- Angelika Szengel, CADD seminars 2017, Charité/FU Berlin
- Marvis Sydow, CADD seminars 2018, Charité/FU Berlin
- Richard Gowers, RDKit UGM hackathon 2019
- Jaime Rodríguez-Guerra, 2020, Volkamer lab, Charité
- Dominique Sydow, 2018-2020, Volkamer lab, Charité

__Talktorial T10__: This talktorial is part of the TeachOpenCADD pipeline described in the [first TeachOpenCADD paper](https://jcheminf.biomedcentral.com/articles/10.1186/s13321-019-0351-x), comprising of talktorials T1-T10.

**Note**: Please run this notebook cell by cell. Running all cells in one is possible also, however, part of the `nglview` 3D representations might be missing.

## Aim of this talktorial

In this talktorial, we use the structural similarity of full proteins and binding sites to predict off-targets, i.e. proteins that are not intended targets of a drug. This may lead to unwanted side effects or enable desired alternate applications of a drug (drug repositioning).
We discuss the main steps for binding site comparison and implement a basic method, i.e. the geometrical variation between structures (the root mean square deviation between two structures).

### Contents in *Theory*

* Off-target proteins
* Computational off-target prediction: binding site comparison
* Pairwise RMSD as simple measure for similarity
* Imatinib, a tyrosine kinase inhibitor

### Contents in *Practical*

* Load and visualize the ligand of interest (Imatinib/STI)
* Get all protein-STI complexes from the PDB
  * Query the PDB
  * Filter the PDB data set
* Visualize the PDB structures
* Align the PDB structures (full protein)
* Get pairwise RMSD (full protein)
* Align the PDB structures (binding site)
* Get pairwise RMSD (binding site)
* Filter out outliers

### References

Binding site superposition + comparison: 

* Binding site comparison reviews: 
  * [<i>Curr. Comput. Aided Drug Des. </i> (2008), <b>4</b>, 209-20](https://www.eurekaselect.com/67606/article/how-measure-similarity-between-protein-ligand-binding-sites)
  * [<i>J. Med. Chem. </i> (2016), <b>9</b>, 4121-51](https://pubs.acs.org/doi/10.1021/acs.jmedchem.6b00078)
* Molecular superposition with Python: `opencadd` package (`structure.superposition` module) ([GitHub repository](https://github.com/volkamerlab/opencadd))
* Wikipedia article on root mean square deviation (RMSD) 
([Wikipedia](https://en.wikipedia.org/wiki/Root-mean-square_deviation_of_atomic_positions)) and structural superposition ([Wikipedia](https://en.wikipedia.org/wiki/Structural_alignment))
* Structural superposition: [Book chapter: Algorithms, Applications, and Challenges of Protein Structure Alignment in *Advances in Protein Chemistry and Structural Biology* (2014), **94**, 121-75](https://www.sciencedirect.com/science/article/pii/B9780128001684000056?via%3Dihub)

Imatinib:  

* Review on Imatinib: [<i>Nat. Rev. Clin. Oncol.</i> (2016), <b>13</b>, 431-46](https://www.nature.com/articles/nrclinonc.2016.41)
* Promiscuity of imatinib: 
[<i>J. Biol.</i> (2009), <b>8</b>, 30](https://jbiol.biomedcentral.com/articles/10.1186/jbiol134)
* [ChEMBL information on Imatinib](https://www.ebi.ac.uk/chembl/compound/inspect/CHEMBL941)
* [PDB information on Imatinib](https://www3.rcsb.org/ligand/STI)
* Side effects of Imatinib
  * [MedFacts Consumer Drug Information](https://www.drugs.com/cdi/imatinib.html)
  * [DrugBank](https://go.drugbank.com/drugs/DB00619)
  * [<i>BMC Struct. Biol.</i> (2009), <b>9</b>](https://bmcstructbiol.biomedcentral.com/articles/10.1186/1472-6807-9-7)

## Theory

### Off-target proteins

An off-target can be any protein which interacts with a drug or (one of) its metabolite(s) without being the designated target protein. 
The molecular reaction caused by the off-target can lead to unwanted side effects, ranging from a rather harmless to an extremely harmful impact. 
Off-targets mainly occur because on- and off-targets share similar structural motifs with each other in their binding site and therefore can bind similar ligands.

### Computational off-target prediction: binding site comparison

Computation-aided prediction of potential off-targets is aimed at minimizing the risk of developing potentially dangerous substances for medical treatment.
There are several algorithmic approaches to assess binding site similarity but they always consist of three main steps:

1. **Binding site encoding**: binding sites are encoded using different descriptor techniques and stored in a target database.
2. **Binding site comparison**: a query binding site is compared with the target database, using different similarity measures.
3. **Target ranking**: targets are ranked based on a suitable scoring approach.

For detailed information on different similarity measures and existing tools, we refer to two excellent reviews on binding site comparison ([<i>Curr. Comput. Aided Drug Des. </i> (2008), <b>4</b>, 209-20](https://www.eurekaselect.com/67606/article/how-measure-similarity-between-protein-ligand-binding-sites) and [<i>J. Med. Chem. </i> (2016), <b>9</b>, 4121-51](https://pubs.acs.org/doi/10.1021/acs.jmedchem.6b00078)).

![Binding site comparison steps](images/binding_site_comparison_steps.png)

Figure 1: Main steps of binding site comparison methods (figure by Dominique Sydow).</div>

### Pairwise RMSD as simple measure for similarity

A simple and straightforward method for scoring the similarity is to use the calculated root mean square deviation (RMSD), which is the square root of the mean of the square of the distances between the atoms of two aligned structures ([Wikipedia](https://en.wikipedia.org/wiki/Root-mean-square_deviation_of_atomic_positions)). 


In order to find the respective atoms that are compared between two structures, they need to be aligned first based on sequence-based or sequence-independent alignment algorithms ([Book chapter: Algorithms, Applications, and Challenges of Protein Structure Alignment in *Advances in Protein Chemistry and Structural Biology* (2014), **94**, 121-75](https://www.sciencedirect.com/science/article/pii/B9780128001684000056?via%3Dihub)).

### Imatinib, a tyrosine kinase inhibitor

Kinases transfer a phosphate group from ATP to proteins, and thereby regulate various cellular processes such as signal transduction, metabolism, and protein regulation.
If these kinases are constitutively active (due to genomic mutations), they can distort regulation processes and cause cancer.
An example for cancer treatment is Imatinib ([<i>Nat. Rev. Clin. Oncol.</i> (2016), <b>13</b>, 431-46](https://www.nature.com/articles/nrclinonc.2016.41)), a small molecule tyrosine kinase inhibitor used to treat cancer, more specifically chronic myeloid leukaemia (CML) and gastrointestinal stromal tumour (GIST). 

Imatinib was shown to be not entirely specific and to target tyrosine kinases other than its main target. This was used for drug repositioning, i.e. Imatinib was approved for alternate cancer types, ([<i>J. Biol.</i> (2009), <b>8</b>, 30](https://jbiol.biomedcentral.com/articles/10.1186/jbiol134)), however can also show unwanted side effects such as signs of an allergic reaction, infection, bleeding, or headache ([MedFacts Consumer Drug Information](https://www.drugs.com/cdi/imatinib.html)).

## Practical

In the following, we will fetch and filter PDB structures that bind Imatinib. We will investigate the structure similarity of Imatinib-binding proteins (those with a solved protein structure). 
The similarity measure used is a pairwise RMSD calculation (as a simple similarity measure), in order to show that this simple method can be used as an initial test for potential off-targets.

In [ ]:
import logging
from pathlib import Path
import random

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import redo
import nglview as nv
import pypdb
from rdkit import Chem
from rdkit.Chem import Draw, AllChem
from rdkit.Chem.Draw import IPythonConsole
from MDAnalysis.analysis import rms
from opencadd.structure.core import Structure
from opencadd.structure.superposition.api import align, METHODS
from opencadd.structure.superposition.engines.mda import MDAnalysisAligner

IPythonConsole.ipython_useSVG=True
logger = logging.getLogger("opencadd")
logger.setLevel(logging.ERROR)

In [ ]:
# Set path to this notebook
HERE = Path(_dh[0])
DATA = HERE / "data"

### Load and visualize the ligand of interest (Imatinib/STI)

The SMILES format for Imatinib (common abbreviation: STI) can be retrieved from e.g. the ChEMBL database 
([ChEMBL: Imatinib](https://www.ebi.ac.uk/chembl/compound/inspect/CHEMBL941)) 
or the PDB database by its common abbreviation STI 
([PDB: STI](https://www3.rcsb.org/ligand/STI)). 
We simply copy the string from the "Isomeric SMILES" entry of the Chemical Component Summary table, and load the ligand here by hand.

In [ ]:
smiles = Chem.MolFromSmiles("CN1CCN(Cc2ccc(cc2)C(=O)Nc2ccc(C)c(Nc3nccc(n3)-c3cccnc3)c2)CC1")
Draw.MolToImage(smiles)

In order to inspect the 3D structure of STI, we use the open source tool `nglview`. 
Before we can view STI in `nglview`, we need to compute its 3D coordinates.

First, we add hydrogen atoms to the molecule, which are not always explicitly denoted in the SMILES format.
Second, we use the distance geometry to obtain initial coordinates for the molecule and optimize the structure of the molecule using the force field UFF (Universal Force Field).

In [ ]:
molecule = Chem.AddHs(smiles)
molecule

In [ ]:
AllChem.EmbedMolecule(molecule)
AllChem.UFFOptimizeMolecule(molecule)
molecule

Now, we are ready to roll in `nglview`! 

In [ ]:
view = nv.show_rdkit(molecule)
view

In [ ]:
view.render_image();

In [ ]:
view._display_image()

### Get all protein-STI complexes from the PDB

#### Query the PDB

We can look up Imatinib/STI in open databases like the PDB and search for proteins that are reported targets. In the PDB, Imatinib is usually abbreviated to STI. We will search for both terms and merge the results in the following. We will use the `pypdb` package to perform the PDB queries.

First, we retrieve all proteins from the PDB that bind the ligand of interest (STI).

In [ ]:
# Set up query dictionary
search_dict = pypdb.make_query("STI")
search_dict

In [ ]:
# Query PDB
pdb_ids1 = pypdb.do_search(search_dict)

print(f"Number of structures connected with STI in the PDB: {len(pdb_ids1)}")
print(*pdb_ids1)

Note that the query results can differ depending on the term used for the query ligand. 

In [ ]:
# Query PDB for proteins bound to the ligand Imatinib
search_dict = pypdb.make_query("Imatinib")  
pdb_ids2 = pypdb.do_search(search_dict)

print(f"Number of structures connected with Imatinib in the PDB: {len(pdb_ids2)}")
print(*pdb_ids2)

We merge both query results and keep only unique entries.

In [ ]:
pdb_ids = list(set(pdb_ids1 + pdb_ids2))
print(f"Number of structures connected with STI/Imatinib in the PDB: {len(pdb_ids)}")

#### Filter the PDB data set

We retrieve meta information on the PDB structures and filter the data set based on the following criteria:

1. Filter by experimental method ("xray").
2. Filter by resolution (equal or lower than 3 &#8491;).
3. Retain only PDB structures with a single chain (for simplicity).
4. Retain only Imatinib-bound structures (e.g. some PDB structures are returned that are associated with Imatinib but not bound to it).
5. Retain only PDB IDs deposited before 2019 (technical detail to make the notebook easier to maintain for us).

Let's get started!

__Get meta information from the PDB.__

In [ ]:
entity_info = [pypdb.get_entity_info(pdb_id) for pdb_id in pdb_ids]
    
# Show the first entry
entity_info[0]

In [ ]:
# Transform list to DataFrame
entity_info = pd.DataFrame(entity_info)

# Show the 5 first rows
print(f"Number of structures: {entity_info.shape[0]}")
entity_info.head()

__1. Filter by experimental method ("xray").__

In [ ]:
entity_info = entity_info[entity_info["Method"] == {"@name": "xray"}]
print(f"Number of structures left: {entity_info.shape[0]}")

__2. Filter by resolution (equal or lower than 3 &#8491;).__

In [ ]:
entity_info = entity_info[entity_info["resolution"].astype(float) <= 3.0]
print(f"Number of structures left: {entity_info.shape[0]}")

__3. Retain only PDB structures with a single chain.__

In [ ]:
# Remove all PDB IDs with multiple entities -- use pandas' apply function
entity_info = entity_info[entity_info["Entity"].apply(lambda x: isinstance(x, dict))]
print(f"Number of structures left: {entity_info.shape[0]}")

# Remove all PDB IDs with multiple chains
entity_info = entity_info[entity_info["Entity"].apply(lambda x: isinstance(x["Chain"], dict))]
print(f"Number of structures left: {entity_info.shape[0]}")

__4. Retain only Imatinib-bound structures.__

In [ ]:
@redo.retriable(attempts=10, sleeptime=2)
def get_ligands(pdb_id):
    """Decorate pypdb.get_ligands so it retries after a failure"""
    return pypdb.get_ligands(pdb_id)

In [ ]:
# Add ligand information to DataFrame
entity_info["ligandInfo"] = entity_info["structureId"].apply(lambda x: get_ligands(x)["ligandInfo"])

# Keep only PBD IDs with ligand information -- "~" negates boolean values
entity_info = entity_info[~entity_info["ligandInfo"].apply(lambda x: x is None)]

# Ligand information is given as dict (if only one ligand) and list, transform everything to list
entity_info["ligandInfo"] = entity_info["ligandInfo"].apply(lambda x: x["ligand"] if isinstance(x["ligand"], list) else [x["ligand"]])

# Keep only PDB IDs with STI ligand
entity_info = entity_info[entity_info["ligandInfo"].apply(lambda x: "STI" in [i["@chemicalID"] for i in x])]
print(f"Number of structures left: {entity_info.shape[0]}")

# After manual visual inspection remove 3GVU (contains 2x STI: not feasible for the automated workflow below)
entity_info = entity_info[entity_info["structureId"] != "3GVU"]
print(f"Number of structures left: {entity_info.shape[0]}")

__5. Retain only PDB IDs deposited before 2019.__

In [ ]:
entity_info.apply(lambda x: int(x["release_date"][-4:]) < 2019, axis=1)
print(f"Number of structures left: {entity_info.shape[0]}")

pdb_ids = entity_info["structureId"].tolist()
print(*pdb_ids)

### Visualize the PDB structures

First, we load all structures in `nglview` for visual inspection of the 3D structure of the protein data set. 

In [ ]:
view = nv.NGLWidget()
for pdb_id in pdb_ids:
    view.add_pdbid(pdb_id)
view

In [ ]:
view.render_image();

In [ ]:
view._display_image()

Though this image is beautifully colorful and curly, it is not informative yet. We align the structures to each other in the next step.

### Align the PDB structures (full protein)

We will use one of our `opencadd` packages (`structure.superposition` module) to guide the structural alignment of the different proteins. The approach we will use is based on superposition guided by sequence alignment provided matched residues. There are other methods in the package, but this simple one will be enough to showcase some similarities.

Note: This approach biases the analysis towards structures with similar sequences. For an automated workflow (where we do not know the sequence or structural similarity of protein pairs) a solution could be to calculate the RMSD based on all three measures and retain the best for further analysis.

First, we show the alignment of all structures to the first structure in the our list.

In [ ]:
# Download PDB 
structures = [Structure.from_pdbid(pdb_id) for pdb_id in pdb_ids]
# Strip solvent and other artifacts of crystallography
proteins = [Structure.from_atomgroup(s.select_atoms("protein")) for s in structures]
# Align proteins
results = align(proteins, method=METHODS["mda"])

In [ ]:
view = nv.NGLWidget()
for protein in proteins:
    view.add_component(protein.atoms)
view

In [ ]:
view.render_image();

In [ ]:
view._display_image()

The structural alignment for many helices is high, whereas lower or poor for other protein parts.

In [ ]:
def calc_rmsd(A, B):
    """
    Calculate RMSD between two structures.
    
    Parameters
    ----------
    A : opencadd.structure.core.Structure
        Structure A.
    B : opencadd.structure.core.Structure
        Structure B.
        
    Returns
    -------
    float
        RMSD value.
    """
    aligner = MDAnalysisAligner()
    selection, _ = aligner.matching_selection(A, B)
    A = A.select_atoms(selection['reference'])
    B = B.select_atoms(selection['mobile'])
    return rms.rmsd(A.positions, B.positions, superposition=False)

In [ ]:
def calc_rmsd_matrix(structures, names):
    """
    Calculate RMSD matrix between a list of structures.
    
    Parameters
    ----------
    structures : list of opencadd.structure.core.Structure
        List of structures.
    names : list of str
        List of structure names.
        
    Returns
    -------
    pandas.DataFrame
        RMSD matrix.
    """
    values = {name: {} for name in names}
    for A, name_i in zip(structures, names):
        for B, name_j, in zip(structures, names):
            if name_i == name_j:
                # If we compare the same structures, RMSD is 0.0 and continue with next iteration
                values[name_i][name_j] = 0.0
                continue
            try:
                # If we already have calculated the RMSD for the pair, reuse RMSD
                values[name_i][name_j] = values[name_j][name_i] 
            except KeyError:
                # If we have not, calculate RMSD
                values[name_i][name_j] = calc_rmsd(A, B)
    df = pd.DataFrame.from_dict(values)
    return df

### Get pairwise RMSD (full protein)

In [ ]:
# TODO: CI
rmsd_matrix = calc_rmsd_matrix(proteins, pdb_ids)
rmsd_matrix

We visualize the results of this RMSD refinement as heatmap.

In [ ]:
# Make sure matplotlib version >= 3.1.2; otherwise you'll get Y-cropped heatmaps
sns.heatmap(rmsd_matrix, linewidths=0, annot=True, square=True, cbar_kws={"label": "RMSD ($\AA$)"}, cmap="Blues");

We cluster the heatmap in order to see protein similarity based on the RMSD refinement.

In [ ]:
def plot_clustermap(rmsd):
    """
    Plot clustered heatmap from import RMSD matrix.
    
    Parameters
    ----------
    rmsd : pandas.DataFrame
        RMSD matrix.
    title : str
        Plot title.
        
    Returns
    -------
    matplotlib.figure.Figure
        Clustered heatmap.
    """
    g = sns.clustermap(rmsd, linewidths=1, annot=True, cbar_kws={"label": "RMSD ($\AA$)"}, cmap="Blues")
    plt.setp(g.ax_heatmap.get_xticklabels(), rotation=0)
    plt.setp(g.ax_heatmap.get_yticklabels(), rotation=0)
    sns.set(font_scale=1.5)
    return plt.gcf()

In [ ]:
plot_clustermap(rmsd_matrix);

The RMSD comparison shows that one protein differs most from the other proteins, i.e. 3FW1. Let's try to understand by checking which proteins we have in our dataset.

Proteins are classified by the chemical reactions that they catalyze with EC (Enzyme Commission) numbers. We will use them here to check the enzymatic groups the proteins belong to.

In [ ]:
# Get EC numbers for PDB IDs from PDB
pdbs_info = [pypdb.get_all_info(pdb_id) for pdb_id in pdb_ids]
ec_numbers = [pdb_info["polymer"]["enzClass"]["@ec"] for pdb_info in pdbs_info]
ec_numbers = {"pdb_id": pdb_ids,  "ec_number": ec_numbers}
ec_numbers = pd.DataFrame(ec_numbers)
ec_numbers

We can see that 3FW1, the human quinone reductase 2 (NQO2), belongs to EC class 1, i.e. oxidoreductases, whereas the other proteins belong to class 2.7, i.e. phosphorus transferases, which contain the tyrosine kinases (EC 2.7.10.2), the designated targets for Imatinib. 3FW1 is a reported off-target "with potential implications for drug design and treatment of chronic myelogenous leukemia in patients" ([<i>BMC Struct. Biol.</i> (2009), <b>9</b>](https://bmcstructbiol.biomedcentral.com/articles/10.1186/1472-6807-9-7)).

### Align PDB structures (binding sites)

So far, we have used the full protein structure for the alignment and RMSD refinement. However, ligands bind only at a protein's binding site. 
Therefore, the similarity of binding sites is a more putative basis for off-target prediction rather than of the similarity of full protein structures. 

We define a binding site of a protein by selecting all residues that are within 10 &#8491; of any ligand atom. These binding site residues are used for alignment and only their C&#593; atoms (protein backbone) are used for the RMSD refinement. Here, we show the alignment of all structures to the first structure in our list.

Note that for the aligner to work, full residues must be selected as part of the distance search, hence the need for the `same residue as (...)` query.

In [ ]:
binding_sites = [Structure.from_atomgroup(s.select_atoms("same residue as (resname STI or (around 10 resname STI))")) 
                 for s in structures]

Let's look at the proteins' binding sites only.

In [ ]:
view = nv.NGLWidget()
for binding_site in binding_sites:
    view.add_component(binding_site.atoms)
view

In [ ]:
view.render_image();

In [ ]:
view._display_image()

Now we align the binding sites.

In [ ]:
results_binding_sites = align(binding_sites, method=METHODS["mda"])

Let's visualize our aligned binding sites.

In [ ]:
view = nv.NGLWidget()
for binding_site in binding_sites:
    view.add_component(binding_site.atoms)
view

In [ ]:
view.render_image();

In [ ]:
view._display_image()

### Get pairwise RMSD (binding sites)

In [ ]:
# TODO: CI
rmsd_matrix_binding_sites = calc_rmsd_matrix(binding_sites, pdb_ids)
rmsd_matrix_binding_sites

We show the clustered heatmap for the RMSD results.

In [ ]:
# Show the pairwise RMSD values as clustered heatmap
plot_clustermap(rmsd_matrix_binding_sites);

What are the key observations in this heatmap?

- As observed already during the *full protein* comparison, also the *binding site* comparison reveals the highest dissimilarity for `3FW1`. Since `3FW1` is the only structure in our dataset representing not a kinase, we are content that our similarity measure was able to spot this. As discussed before, the human quinone reductase 2 (`3FW1`) is a reported off-target for Imatinib.
- Also the kinase `1XBB` shows a higher dissimilarity compared to the other kinases. This can be explained by `1XBB` being resolved in a different conformation (DFG-in) than the other kinases (DFG-out). The DFG motif is an important structural element in kinases, defining whether a kinase is active (DFG-in conformation) or inactive (DFG-out conformation). Note: DFG conformations of kinase structures can be looked up e.g. in the [KLIFS database](www.klifs.net).
- The remaining structures are comparatively similar to each other, which we would expect since they all represent kinases in the overall same conformation.

Note that RMSD values as calculated here are dependent on the residue selection (binding site definition) and the quality of the a priori sequence alignment.

### Filter out outliers

For a cleaner depiction of the most similar structures, we can filter `3FW1` and `1XBB` out and re-run the binding site comparison.

In [ ]:
filtered_structures = []
filtered_pdb_ids = []
for name, structure in zip(pdb_ids, structures):
    if name not in ("3FW1", "1XBB"):
        filtered_structures.append(structure)
        filtered_pdb_ids.append(name)

In [ ]:
filtered_binding_sites = [Structure.from_atomgroup(s.select_atoms("same residue as (resname STI or (around 10 resname STI))")) 
                          for s in filtered_structures]

In [ ]:
view = nv.NGLWidget()
for binding_site in filtered_binding_sites:
    view.add_component(binding_site.atoms)
view

In [ ]:
view.render_image();

In [ ]:
view._display_image()

In [ ]:
filtered_results_binding_sites = align(filtered_binding_sites, method=METHODS["mda"])

In [ ]:
view = nv.NGLWidget()
for binding_site in filtered_binding_sites:
    view.add_component(binding_site.atoms)
view

In [ ]:
view.render_image();

In [ ]:
view._display_image()

In [ ]:
filtered_rmsd_matrix_bs = calc_rmsd_matrix(filtered_binding_sites, filtered_pdb_ids)
filtered_rmsd_matrix_bs

In [ ]:
plot_clustermap(filtered_rmsd_matrix_bs);

Much better!

## Discussion

In this talktorial, we have used a simple comparison approach, i.e. sequence alignment and subsequent RMSD refinement of (i) full proteins and (ii) binding sites, to assess the similarity and dissimilarity of a small set of structures showing Imatinib-binding proteins. 
In our data set, we were able to spot an off-target for Imatinib (highest dissimilarity) and a kinase resolved in a different conformation compared to the rest of the kinases (also with a relatively high dissimilarity). 
Given our simple approach, we are content with these results!

In a real case scenario, off-targets for Imatinib would be predicted by comparing the binding site of an intended target of Imatinib (a tyrosine kinase) with a large database of resolved structures (PDB). 
Since this results in the comparison of sequences also with low similarity, more sophisticated methods should be invoked
that use a sequence-independent alignment algorithm and that include the physico-chemical properties of the binding site.

## Quiz

1. Explain the terms on- and off-targets of a drug.
2. Explain why binding site similarity can be used to find off-targets based on a query target.
3. Discuss how useful the RMSD value of (i) full proteins and (ii) protein binding sites is for off-target prediction.
4. Think of alternate approaches to encode binding site information.